# About: LDAP認証の設定

---

MoodleのLDAP認証プラグインの設定を行います。

## 概要

### 前提条件

LDAP認証を設定するために必要となる準備事項を以下に示します。

* LDAPサーバにアクセスするための認証情報など
* CA証明書（TLS接続を行う場合かつ、追加が必要な場合）

## LDAPサーバに接続するための設定

LDAPサーバにTLS接続を行う場合、以下の設定が必要となることがあります。

* CA証明書の配置
* LDAPサーバの名前解決に関する設定

追加の設定が不要な場合はこの章の実行をスキップしてください。

### 設定対象となるMoodle環境の指定

設定対象となるMoodle環境にアクセスするためのAnsibleのグループ名を指定してください。

In [ ]:
# (例)
# target_group = 'Moodle'

target_group =

Ansibleでアクセスできることを確認します。

In [ ]:
!ansible {target_group} -m ping

### CA証明書の追加

Moodle環境にCA証明書を追加します。

次のセルにCA証明書の内容を指定してください。

In [ ]:
# (例)
# ca_certificate = '''-----BEGIN CERTIFICATE-----
# MIIC0zCCAlmgAwIBAgIUCfQ+m0pgZ/BjYAJvxrn/bdGNZokwCgYIKoZIzj0EAwMw
# (中略)
# 8FdfWPypvA==
# -----END CERTIFICATE-----'''

ca_certificate = '''-----BEGIN CERTIFICATE-----

-----END CERTIFICATE-----'''

MoodleコンテナにCA証明書を配置します。

In [ ]:
from tempfile import TemporaryDirectory
from pathlib import Path

with TemporaryDirectory() as workdir:
    ca_pem = Path(workdir) / 'ldap-ca.pem'
    with ca_pem.open(mode='w') as f:
        f.write(ca_certificate)
    !ansible {target_group} -b -m copy -a \
        'src={ca_pem} dest=/srv/moodle/moodle/conf/ca-trust/'

CA証明書を追加します。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    docker compose exec moodle \
    update-ca-trust extract'

### LDAPサーバの名前解決に関する設定

MoodleコンテナからLDAPサーバのホスト名の名前解決を行えるように設定を追加します。

MoodleコンテナからLDAPサーバの名前解決を行うための設定を`docker-compose.yml`に追加するには、以下に示す２つの設定方法があります。

* [DNSサーバの設定を追加する](https://docs.docker.com/compose/compose-file/#dns)
* [コンテナ内の/etc/hostsにエントリを追加する](https://docs.docker.com/compose/compose-file/#extra_hosts)

DNSサーバの設定を`docker-compose.yml`に追加する場合の例を以下に示します。この例ではIPアドレス `8.8.8.8`のDNSサーバをMoodleコンテナに追加しています。

```
  moodle:
    dns:
      - 8.8.8.8 # DNSサーバのIPアドレスを指定する
```

Moodleコンテナの`/etc/hosts` にエントリを追加する場合の例を以下に示します。この例ではホスト名 `ldap.example.org`、IPアドレス`192.168.10.100`のエントリをMoodleコンテナの`/etc/hosts`に追加しています。

```
  moodle:
    extra_hosts:
      - "ldap.example.org:192.168.10.100"
```

実際に`docker-compose.yml`を編集するために、Moodle環境からローカル環境に `docker-compose.yml` ファイルの取得を行います。実行結果に表示されているリンクをクリックすることで、ローカル環境に取得した `docker-compose.yml` を編集することができます。

> ファイルの編集後は**必ず**、メニューの[File]-[Save]を選択してファイルの保存を行ってください。

In [ ]:
%run scripts/edit_conf.py
fetch_docker_compose(target_group)

`docker-compose.yml` を編集した後に、次のセルを実行すると編集の前後での差分を確認することができます。

In [ ]:
show_local_docker_compose_diff(target_group)

DNSサーバの設定を追加した場合の差分の表示例を以下に示します。

```
--- docker-compose.yml.orig
+++ docker-compose.yml
@@ -25,6 +25,8 @@
       - --explicit_defaults_for_timestamp=true
       - --log-error=/var/log/mysql/error.log
   moodle:
+    dns:
+      - 8.8.8.8
     image: harbor.vcloud.nii.ac.jp/vcp/moodle-simple:3.9
     container_name: moodle
     restart: always
```

`/etc/hosts`のエントリを追加する設定を行った場合の差分の表示例を以下に示します。

```
--- docker-compose.yml.orig
+++ docker-compose.yml
@@ -25,6 +25,8 @@
       - --explicit_defaults_for_timestamp=true
       - --log-error=/var/log/mysql/error.log
   moodle:
+    extra_hosts:
+      - "ldap.example.org:192.168.10.100"
     image: harbor.vcloud.nii.ac.jp/vcp/moodle-simple:3.9
     container_name: moodle
     restart: always
```

ローカル環境の`docker-compose.yml`をMoodle環境に配置して、さらに変更内容を変更するためにMoodleコンテナの再起動を行います。

In [ ]:
apply_docker_compose(target_group)

## LDAP認証プラグインの設定

MoodleでLDAP認証行うための設定を行います。

### LDAP認証プラグインの有効化

まずLDAP認証プラグインを有効にします。管理者ユーザでログインして[サイト管理]-[プラグイン]-[認証管理]を選択し認証プラグインの一覧を表示してください。以下のような画面が表示されます。

![認証管理画面](images/moodle-211-01.png)

図の「LDAPサーバ」の行の `Yes` の列にあるアイコンを選択してLDAP認証プラグインを有効化してください。

### LDAPサーバの設定

「LDAPサーバ」の「設定」のリンクを選択すると以下のような設定画面が表示されます。

![ldap設定画面](images/moodle-211-02.png)

「ホストURL」などにLDAPサーバのアドレスなどを設定してください。

また、同じ画面の「バインド設定」でLDAPサーバに接続するための識別名、パスワードも必要に応じて設定してください。

![バインド設定](images/moodle-211-03.png)

最後に設定画面の最下部にある「変更を保存する」ボタンを選択してください。

### LDAPサーバへの接続確認

認証プラグインの一覧画面にある「設定テスト」リンクを選択すると、現在の設定でLDAPサーバに接続できるかをテストすることができます。

![ldap設定テスト](images/moodle-211-04.png)

LDAPサーバへの接続が成功すると以下のような表示となります。

![ldap設定結果](images/moodle-211-05.png)